#### Reference:
https://www.youtube.com/watch?v=cO5g5qLrLSo&t=218s

tensorflow==2.5.0
gym
keras-rl2

In [1]:
import gym
import random

In [2]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0]
actions = env.action_space.n

In [3]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])

        steps = env.step(action)
        n_state = steps[0]
        reward = steps[1]
        done = steps[2]
        info = steps[3]

        score += reward
    
    print(f"Episode: {episode} Score: {score}")


Episode: 1 Score: 12.0
Episode: 2 Score: 18.0
Episode: 3 Score: 9.0
Episode: 4 Score: 23.0
Episode: 5 Score: 28.0
Episode: 6 Score: 17.0
Episode: 7 Score: 9.0
Episode: 8 Score: 19.0
Episode: 9 Score: 16.0
Episode: 10 Score: 10.0


#### Creating a Deep Learning Model to Improve the Scores of the Reinforcement Learning Game

In [10]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [11]:
def generate_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(actions, activation="linear"))
    return model

model = generate_model(states, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


#### Building Agents using Keras-RL

In [12]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [13]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(
        model=model,
        memory=memory,
        policy=policy,
        nb_actions=actions,
        nb_steps_warmup=10,
        target_model_update=1e-2
    )
    return dqn

dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=["mae"])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 22:04 - reward: 1.0000

C:\Users\robot\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
c:\Users\robot\anaconda3\envs\rfenv\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 156s 16ms/step - reward: 1.0000
101 episodes - episode_reward: 98.129 [9.000, 200.000] - loss: 3.124 - mae: 19.770 - mean_q: 40.099

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 158s 16ms/step - reward: 1.0000
55 episodes - episode_reward: 180.564 [149.000, 200.000] - loss: 4.218 - mae: 39.488 - mean_q: 80.362

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 173s 17ms/step - reward: 1.0000
54 episodes - episode_reward: 186.037 [160.000, 200.000] - loss: 3.062 - mae: 40.338 - mean_q: 82.046

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 172s 17ms/step - reward: 1.0000
53 episodes - episode_reward: 190.245 [159.000, 200.000] - loss: 2.285 - mae: 37.143 - mean_q: 75.324

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 174s 17ms/step - reward: 1.0000
done, took 832.645 seconds


In [14]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history["episode_reward"]))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [16]:
# visualizing our model
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


: 

#### Saving Model and Reloading Wights, Agent from Memory

In [ ]:
dqn.save_weights("./models_folder/cartpole/dqn_weights.h5f", overwrite=True)

In [ ]:
# rebuilding environment and testing model
env = gym.make("CartPole-v0")
actions = env.action_space.n
states = env.observation_space.shape[0]
model = generate_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=["mae"])

dqn.load_weights("./models_folder/cartpole/dqn_weights.h5f")

_ = dqn.test(env, nb_episodes=1, visualize=True)